In [1]:
#!/usr/bin/env python
# coding: utf-8

import os, sys
import logging

import numpy as np
import h5netcdf

import xarray as xr
import xarray.ufuncs as xu
import xrft
import pandas as pd
from scipy.signal import convolve2d, detrend

from matplotlib import pyplot as plt

import cartopy.crs as ccrs
import cartopy

plt.rc("figure", figsize=(12,10))
plt.rc("font", size=14)

from dask.distributed import Client, LocalCluster
#
# Initialisation d'un cluster de 32 coeurs
cluster = LocalCluster(processes=False, n_workers=1, threads_per_worker=4, silence_logs='error')
client = Client(cluster)
client

/home/durandy/miniconda3/envs/towel/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35313 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://137.129.155.67:35313/status,
Dashboard: http://137.129.155.67:35313/status,Workers: 1
Total threads: 4,Total memory: 78.61 GiB
Status: running,Using processes: False
Comm: inproc://137.129.155.67/39524/1,Workers: 1
Dashboard: http://137.129.155.67:35313/status,Total threads: 4
Started: Just now,Total memory: 78.61 GiB
Comm: inproc://137.129.155.67/39524/4,Total threads: 4
Dashboard: http://137.129.155.67:41697/status,Memory: 78.61 GiB
Nanny: None,


In [2]:
path = "/home/durand/Documents/OLR/"

indir_data = path + 'Anomaly/'
outdir_TF = path + 'TF2D/'

indir_data = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/RAW_ANOMALY/TCWV/'
outdir_data = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/FILTERED_ANOMALY/TCWV/'

var_file = 'anom_tcwv_brut_ERA5_3H_'

addDay = 180
spd = 8

In [3]:
rlat = 0
pi    = np.pi
radius = 6.37122e06    # [m]   average radius of earth
g     = 9.80665        # [m/s] gravity at 45 deg lat used by the WMO
omega = 7.292e-05      # [1/s] earth's angular vel
ll    = 2.*pi*radius*np.cos(np.abs(rlat))
Beta  = 2.*omega*np.cos(np.abs(rlat))/radius
fillval = 1e20

In [4]:
###################################################################################
def createArray(year) :
    _ds_m1 = xr.open_mfdataset(indir_data+'*'+var_file+'*'+str(year-1)+'.nc', chunks = {'lat' : 1}, parallel=True)
    _ds_m1 = _ds_m1.isel(time = slice(addDay*spd,None))
    _ds = xr.open_mfdataset(indir_data+'*'+var_file+'*'+str(year)+'.nc', chunks = {'lat' : 1}, parallel=True)
    _ds1 = xr.open_mfdataset(indir_data+'*'+var_file+'*'+str(year+1)+'.nc', chunks = {'lat' : 1}, parallel=True)
    _ds1 = _ds1.isel(time = slice(None,addDay*spd))

    ds = xr.concat([_ds_m1,_ds,_ds1], dim='time', coords='minimal', compat='override')
    
    return ds

In [5]:
def split_hann_taper(series_length, fraction):
    """Implements `split cosine bell` taper of length series_length where only fraction of points are tapered (combined on both ends).
    
    This returns a function that tapers to zero on the ends. To taper to the mean of a series X:
    XTAPER = (X - X.mean())*series_taper + X.mean()
    """
    npts = int(np.rint(fraction * series_length))  # total size of taper
    taper = np.hanning(npts)
    series_taper = np.ones(series_length)
    series_taper[0:npts//2+1] = taper[0:npts//2+1]
    series_taper[-npts//2+1:] = taper[npts//2+1:]
    return series_taper, taper


def convolvePosNeg(arr, k, dim, boundary_index):
    """Apply convolution of (arr, k) excluding data at boundary_index in dimension dim.
    
    arr: numpy ndarray of data
    k: numpy ndarray, same dimension as arr, this should be the kernel
    dim: integer indicating the axis of arr to split
    boundary_index: integer indicating the position to split dim
    
    Split array along dim at boundary_index;
    perform convolution on each sub-array;
    reconstruct output array from the two subarrays;
    the values of output at boundary_index of dim will be same as input.
    
    `convolve2d` is `scipy.signal.convolve2d()`
    """
    # arr: numpy ndarray
    oarr = arr.copy()  # maybe not good to make a fresh copy every time?
    # first pass is [0 : boundary_index)
    slc1 = [slice(None)] * arr.ndim
    slc1[dim] = slice(None, boundary_index)
    arr1 = arr[tuple(slc1)]
    ans1 = convolve2d(arr1, k, boundary='symm', mode='same')
    # second pass is [boundary_index+1, end]
    slc2 = [slice(None)] * arr.ndim
    slc2[dim] = slice(boundary_index+1,None)
    arr2 = arr[tuple(slc2)]
    ans2 = convolve2d(arr2, k, boundary='symm', mode='same')
    # fill in the output array
    oarr[tuple(slc1)] = ans1
    oarr[tuple(slc2)] = ans2
    return oarr

In [6]:
def kelvinfilter(da, fmin=None, fmax=None, kmin=None, kmax=None, hmin=None, hmax=None, rlat = 0):
    """kelvin wave filterhttp://137.129.155.67:8888/notebooks/TF/ANOMALY/TF_ANOMALY_2D_OLR_ANOMALY_FILTER.ipynb#
    Arguments:
       'fmin/fmax' -- unit is cycle per day
       'kmin/kmax' -- zonal wave number
       'hmin/hmax' --equivalent depth
    """
    pi    = np.pi
    radius = 6.37122e06    # [m]   average radius of earth
    g     = 9.80665        # [m/s] gravity at 45 deg lat used by the WMO
    omega = 7.292e-05      # [1/s] earth's angular vel
    ll    = 2.*pi*radius*np.cos(np.abs(rlat))
    Beta  = 2.*omega*np.cos(np.abs(rlat))/radius
    fillval = 1e20
    
    knum = da.freq_lon
    freq = da.freq_time

    # filtering ############################################################
    mask = da.copy()
    #wavenumber cut-off
    if kmin != None:
        mask = mask.where((da.freq_lon > kmin) & (da.freq_lon < kmax), drop = False)
    
    #frequency cutoff
    if fmin != None:
        mask = mask.where((da.freq_time > fmin) & (da.freq_time < fmax), drop = False)

    #dispersion filter
    if hmin != None:
        c = np.sqrt(g*hmin)
        omega = -2.*pi*freq/24./3600. / np.sqrt(Beta*c) #adusting day^-1 to s^-1
        k     = 1/((radius)/(knum-1)) * np.sqrt(c/Beta)         #adusting wavenumber to m
        mask = mask.where(lambda da: -(omega - k) <= 0., drop = False)
    if hmax != None:
        c = np.sqrt(g*hmax)
        omega = -2.*pi*freq/24./3600. / np.sqrt(Beta*c) #adusting day^-1 to s^-1
        k     = 1/((radius)/(knum+1)) * np.sqrt(c/Beta)         #adusting wavenumber to m
        mask = mask.where(lambda da: -(omega - k) >= 0., drop = False)

    _varhat = mask.compute()
    _varhat['freq_lon'] = freq_lon_Save
    _varhat['freq_time'] = freq_time_Save
    filterd = xrft.ifft(_varhat.fillna(0.), dim = ['freq_time','freq_lon'], 
                                  true_phase=False, true_amplitude=True) # Signal in direct space
    filterd['time'] = ds['time']
    return mask, filterd

def mjofilter(da, fmin=None, fmax=None, kmin=None, kmax=None) :
    """kelvin wave filter
    Arguments:
       'fmin/fmax' -- unit is cycle per day
       'kmin/kmax' -- zonal wave number
       'hmin/hmax' --equivalent depth
    """
    
    knum = da.freq_lon
    freq = da.freq_time

    # filtering ############################################################
    mask = da.copy()
    #wavenumber cut-off
    if kmin != None:
        mask = mask.where((da.freq_lon > kmin) & (da.freq_lon < kmax), drop = False)
    
    #frequency cutoff
    if fmin != None:
        mask = mask.where((da.freq_time > fmin) & (da.freq_time < fmax), drop = False)

    _varhat = mask.compute()
    _varhat['freq_lon'] = freq_lon_Save
    _varhat['freq_time'] = freq_time_Save
    filterd = xrft.ifft(_varhat.fillna(0.), dim = ['freq_time','freq_lon'], 
                                  true_phase=False, true_amplitude=True)  # Signal in direct space
    filterd['time'] = ds['time']
    return mask, filterd

def erfilter(da, fmin=None, fmax=None, kmin=None, kmax=None, hmin=None, hmax=None, n=1, rlat = 0):
    """equatorial wave filter
    Arguments:
       'fmin/fmax' -- unit is cycle per day
       'kmin/kmax' -- zonal wave number
       'hmin/hmax' -- equivalent depth
       'n'         -- meridional mode number
    """

    if n <=0 or n%1 !=0:
        print ("n must be n>=1 integer")
        sys.exit()

    knum = da.freq_lon
    freq = da.freq_time

    
    # filtering ############################################################
    mask = da.copy()
    #wavenumber cut-off
    if kmin != None:
        mask = mask.where((da.freq_lon > kmin) & (da.freq_lon < kmax), drop = False)
    
    #frequency cutoff
    if fmin != None:
        mask = mask.where((da.freq_time > fmin) & (da.freq_time < fmax), drop = False)
        
        
    # filtering ############################################################
    if hmin != None:
        c = np.sqrt(g*hmin)
        omega = -2.*pi*freq/24./3600. / np.sqrt(Beta*c)           #adusting day^-1 to s^-1
        k     = 1/((radius)/(knum+1)) * np.sqrt(c/Beta)                          #adusting ^2pia to ^m               
        mask = mask.where(lambda da: -(omega*(k**2 + (2*n+1)) + k) <= 0., drop = False)   
    if hmax != None:
        c = np.sqrt(g*hmax)
        omega = -2.*pi*freq/24./3600. / np.sqrt(Beta*c)           #adusting day^-1 to s^-1
        k     = 1/((radius)/(knum-1)) * np.sqrt(c/Beta)   
        mask = mask.where(lambda da: -(omega*(k**2 + (2*n+1)) + k) >= 0., drop = False)             

    _varhat = mask.compute()
    _varhat['freq_lon'] = freq_lon_Save
    _varhat['freq_time'] = freq_time_Save
    filterd = xrft.ifft(_varhat.fillna(0.), dim = ['freq_time','freq_lon'], 
                                  true_phase=False, true_amplitude=True) # Signal in direct space
    filterd['time'] = ds['time']
    return mask, filterd

def mrgfilter(da, fmin=None, fmax=None, kmin=None, kmax=None, hmin=None, hmax=None,):
    """mixed Rossby gravity wave
    Arguments:
       'fmin/fmax' -- unit is cycle per day
       'kmin/kmax' -- zonal wave number. negative is westward, positive is
                      eastward
       'hmin/hmax' -- equivalent depth
    """
    
    knum = da.freq_lon
    freq = da.freq_time
    
        
    # filtering ############################################################
    mask = da.copy()
    #wavenumber cut-off
    if kmin != None:
        mask = mask.where((da.freq_lon > kmin) & (da.freq_lon < kmax), drop = False)
    
    #frequency cutoff
    if fmin != None:
        mask = mask.where((da.freq_time > fmin) & (da.freq_time < fmax), drop = False)
        
        
    #dispersion filter
    mask = mask.where((da.freq_lon <= -1.), drop = False)
    if hmin != None:
        c = np.sqrt(g*hmin)
        omega = -2.*pi*freq/24./3600. / np.sqrt(Beta*c)           #adusting day^-1 to s^-1
        k     = 1/((radius)/(knum-1)) * np.sqrt(c/Beta)                         #adusting ^2pia to ^m               
        mask = mask.where(lambda da: -(omega**2 - k*omega - 1) <= 0., drop = False) 
    if hmax != None:
        c = np.sqrt(g*hmax)
        omega = -2.*pi*freq/24./3600. / np.sqrt(Beta*c)           #adusting day^-1 to s^-1
        k     = 1/((radius)/(knum+1)) * np.sqrt(c/Beta)                 #adusting ^2pia to ^m               
        mask = mask.where(lambda da: -(omega**2 - k*omega - 1) >= 0., drop = False) 
  
    _varhat = mask.compute()
    _varhat['freq_lon'] = freq_lon_Save
    _varhat['freq_time'] = freq_time_Save
    filterd = xrft.ifft(_varhat.fillna(0.), dim = ['freq_time','freq_lon'], 
                                  true_phase=False, true_amplitude=True) # Signal in direct space
    filterd['time'] = ds['time']
    return mask, filterd

def igfilter(da, fmin=None, fmax=None, kmin=None, kmax=None, hmin=None, hmax=None, n=1):
    """n>=1 inertio gravirt wave filter. default is n=1 WIG.
    Arguments:
       'fmin/fmax' -- unit is cycle per day
       'kmin/kmax' -- zonal wave number. negative is westward, positive is
                      eastward
       'hmin/hmax' -- equivalent depth
       'n'         -- meridional mode number
    """

    # filtering ############################################################
    mask = da.copy()
    #wavenumber cut-off
    if kmin != None:
        mask = mask.where((da.freq_lon > kmin) & (da.freq_lon < kmax), drop = False)
    
    #frequency cutoff
    if fmin != None:
        mask = mask.where((da.freq_time > fmin) & (da.freq_time < fmax), drop = False)
        
    knum = da.freq_lon
    freq = da.freq_time
    #dispersion filter
    if hmin != None:
        c = np.sqrt(g*hmin)
        omega = -2.*pi*freq/24./3600. / np.sqrt(Beta*c)           #adusting day^-1 to s^-1
        k     = 1/((radius)/(knum-1)) * np.sqrt(c/Beta)                   #adusting ^2pia to ^m               
        mask = mask.where(lambda da: -(omega**2 - k**2 - (2*n+1)) <= 0., drop = False) 
    if hmax != None:
        c = np.sqrt(g*hmax)
        omega = -2.*pi*freq/24./3600. / np.sqrt(Beta*c)           #adusting day^-1 to s^-1
        k     = 1/((radius)/(knum-1)) * np.sqrt(c/Beta)                      #adusting ^2pia to ^m               
        mask = mask.where(lambda da: -(omega**2 - k**2 - (2*n+1)) >= 0., drop = False) 
        
        
    _varhat = mask.compute()
    _varhat['freq_lon'] = freq_lon_Save
    _varhat['freq_time'] = freq_time_Save
    filterd = xrft.ifft(_varhat.fillna(0.), dim = ['freq_time','freq_lon'], 
                                  true_phase=False, true_amplitude=True) # Signal in direct space
    filterd['time'] = ds['time']
    return mask, filterd

In [7]:
def deletebug(da):
    da = da.sel(time = str(f))
    _da = da.real.values
    _da = np.roll( _da, 180, axis = 2)

    da = xr.DataArray(_da,
                       dims=("time","lat","lon"), 
                       coords={"time":da.time,
                               "lat":da.lat,
                               "lon":da.lon})
    da['time'] = date_Re
      
    return da

In [8]:
filenames = np.arange(2001,2019)
for f in filenames:
    date_Re = pd.date_range(start = (str(f) + '-01-01'), end = (str(f + 1) + '-01-01'), freq = '3H', closed = 'left')

    ds = createArray(f)
    ds = ds.rename({'latitude':'lat','longitude':'lon'})
    x_wintap = ds['tcwv_ano'].chunk({"time" : -1, "lat": 1})

    tcwvhat = xrft.fft(x_wintap, detrend='linear',
                dim=['time','lon'], true_phase=False, true_amplitude=True)

    _tcwvhat = tcwvhat.copy()

    #####################################
    ### We save freq_lon in ° and freq_time in second, because we use wavenumber and cycle by day for filtering
    freq_lon_Save = tcwvhat['freq_lon']
    freq_time_Save = tcwvhat['freq_time']
    #####################################
    wavenumber = np.zeros(_tcwvhat.freq_lon.size)
    for i in range( _tcwvhat.freq_lon.size):
        j= - int(360/2) + i
        wavenumber[i] = _tcwvhat.freq_lon[int(360/2)+j]*360 + 1
    #     print(wavenumber[i])

    _tcwvhat['freq_lon'] = wavenumber
    _tcwvhat['freq_time'] = tcwvhat.freq_time*86400
    #####################################
    # Kelvin Filtering
    """
    kelvin: (Straub Kiladis, Kiladis 2009)
      tMin = 2.5
      tMax = 17.
      kMin = 1
      kMax = 14
      hMin = 8
      hMax = 90
    """

    _dahat_Kev, da_Kevfilter = kelvinfilter(_tcwvhat, fmin = -1/2.5, fmax = -1/17, kmax = 14, kmin = 1, hmin=8, hmax=90)

    ######################################
    # Rossby Filtering
    """
    n=1 Eq. Rossby  Kiladis et al 2009
    tMin = 9.7
    tMax = 72.
    kMin = -10
    kMax = -1
    hMin = 8 
    hMax = 90
    """
    _dahat_Ross, da_Rossfilter = erfilter(_tcwvhat, fmin = -1/9.7, fmax = -1/72, kmax = -1, kmin = -10., hmin=8, hmax=90)

    #########################################
    # MJO Filtering
    """
    MJO
    waveName = "MJO"   ; kiladis et al. 2009
    tMin = 30.
    tMax = 96.
    kMin = 0
    kMax = 9  # 5 is also possible
    hMin = mis
    hMax = mis
    """
    _dahat_MJO, da_MJOfilter = mjofilter(_tcwvhat, fmin = -1/30, fmax= -1/96, kmax = 9, kmin = 0)

    #############################################
    # MRG Filtering
    """
    MRG
    tMin = 3.
    tMax = 8.
    kMin = -10
    kMax = -1
    hMin = 8
    hMax = 90
    """
    _dahat_MRG, da_MRGfilter = mrgfilter(_tcwvhat, fmin = -1/3, fmax = -1/8, kmax = -1, kmin = -10, hmin = 8, hmax = 90)


    ################################################
    # IGW Filtering
    """
    EIG  (Wheeler Kiladis 1998)    
    tMin = 1.
    tMax = 5.
    kMin = 14
    kMax = 1
    hMin = 12  
    hMax = 50


    WIG  (Wheeler Kiladis 1998)
    tMin = 1.25
    tMax = 2.
    kMin = 14
    kMax = 1
    hMin = 12 
    hMax = 50
    """
    _dahat_EIG, da_EIGfilter = igfilter(_tcwvhat, fmin = -1, fmax = -1/5, kmin=1, kmax=14, hmin=12)
    _dahat_WIG, da_WIGfilter = igfilter(_tcwvhat, fmin = -1/1.25, fmax = -1/2, kmin=-14, kmax=-1, hmin=12, hmax=50)

    ######################################################
    # TD filtering
    """
    TD (Easterly wave) : 
    tMin = 2.5
    tMax = 7.
    kMin = -20
    kMax = -6
    hMin = mis  ==> Min curve = up curve of MRG Domain
    hMax = mis
    """
    _dahat_TD, da_TDfilter = mrgfilter(_tcwvhat, fmin = -1/2.5, fmax = -1/7, kmax = -6, kmin = -20, hmin = 12)
    # #######################################################
    # #######################################################
    # delete bug 
    da_Kevfilter = deletebug(da_Kevfilter)
#     da_Rossfilter = deletebug(da_Rossfilter)
#     da_MJOfilter = deletebug(da_MJOfilter)
#     da_MRGfilter = deletebug(da_MRGfilter)
#     da_EIGfilter = deletebug(da_EIGfilter)
#     da_WIGfilter = deletebug(da_WIGfilter)
#     da_TDfilter = deletebug(da_TDfilter)

#     #######################################################
#     #######################################################
#     # NetCDF save
#     var = 'TCWV'
#     ds_NetCdf = da_Kevfilter.to_dataset(name = var + '_Kelvin')
#     ds_NetCdf[var + '_Rossby'] = da_Rossfilter
#     ds_NetCdf[var + '_MJO'] = da_MJOfilter
#     ds_NetCdf[var + '_MRG'] = da_MRGfilter
#     ds_NetCdf[var + '_EIG'] = da_EIGfilter
#     ds_NetCdf[var + '_WIG'] = da_WIGfilter
#     ds_NetCdf[var + '_TD'] = da_TDfilter

#     ds_NetCdf = ds_NetCdf.sel(time = str(f))
#     ds_NetCdf.to_netcdf(outdir_data + 'TCWV_ERA5_3H_' + str(f) +'.nc', unlimited_dims='time')

KeyError: 'TCWV_ano'

In [9]:
ds

<xarray.Dataset>
Dimensions:   (lat: 321, lon: 1440, time: 5848)
Coordinates:
  * lat       (lat) float32 40.0 39.75 39.5 39.25 ... -39.25 -39.5 -39.75 -40.0
  * lon       (lon) float32 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
  * time      (time) datetime64[ns] 2000-06-29 ... 2002-06-29T21:00:00
Data variables:
    tcwv_ano  (time, lat, lon) float64 dask.array<chunksize=(1488, 321, 1440), meta=np.ndarray>

In [ ]:
plt.rc("figure", figsize=(16,4))
plt.rc("font", size=14)
ccrs.PlateCarree()

map_proj = ccrs.Mercator(central_longitude=0.0, 
                         min_latitude=-30.0, 
                         max_latitude=30.0, 
                         globe=None)

fig, axis = plt.subplots(
    1, 1, subplot_kw={'projection': map_proj})

da_WIGfilter.real.isel(time = int(5368/2)).plot.contourf(ax=axis, levels = 101,
                    transform=ccrs.PlateCarree())

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

In [ ]:
plt.rc("figure", figsize=(16,4))
plt.rc("font", size=14)
ccrs.PlateCarree()

map_proj = ccrs.Mercator(central_longitude=0.0, 
                         min_latitude=-30.0, 
                         max_latitude=30.0, 
                         globe=None)

power = da_MJOfilter
for i in range(20) :
    fig, axis = plt.subplots(
        1, 1, subplot_kw={'projection': map_proj})

    power.isel(time = i*8*4).plot.contourf(ax=axis, levels = 101,
                        transform=ccrs.PlateCarree())

    axis.coastlines()
    gl = axis.gridlines(draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False
    plt.show()

In [ ]:
plt.rc("figure", figsize=(16,4))
plt.rc("font", size=14)
ccrs.PlateCarree()

map_proj = ccrs.Mercator(central_longitude=0.0, 
                         min_latitude=-30.0, 
                         max_latitude=30.0, 
                         globe=None)

fig, axis = plt.subplots(
    1, 1, subplot_kw={'projection': map_proj})

da_Kevfilter.real.sel(time ='01-01-2008T00').plot.contourf(ax=axis, levels = 101,
                    transform=ccrs.PlateCarree())

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

In [ ]:
plt.rc("figure", figsize=(16,4))
plt.rc("font", size=14)
ccrs.PlateCarree()

map_proj = ccrs.Mercator(central_longitude=0.0, 
                         min_latitude=-30.0, 
                         max_latitude=30.0, 
                         globe=None)

fig, axis = plt.subplots(
    1, 1, subplot_kw={'projection': map_proj})

da_Rossfilter.real.isel(time = int(5368/2)).plot.contourf(ax=axis, levels = 101,
                    transform=ccrs.PlateCarree())

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

In [ ]:
plt.rc("figure", figsize=(16,4))
plt.rc("font", size=14)
ccrs.PlateCarree()

map_proj = ccrs.Mercator(central_longitude=0.0, 
                         min_latitude=-30.0, 
                         max_latitude=30.0, 
                         globe=None)

fig, axis = plt.subplots(
    1, 1, subplot_kw={'projection': map_proj})

da_MRGfilter.real.isel(time = int(5368/2)).plot.contourf(ax=axis, levels = 101,
                    transform=ccrs.PlateCarree())

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

In [ ]:
plt.rc("figure", figsize=(16,4))
plt.rc("font", size=14)
ccrs.PlateCarree()

map_proj = ccrs.Mercator(central_longitude=0.0, 
                         min_latitude=-30.0, 
                         max_latitude=30.0, 
                         globe=None)

fig, axis = plt.subplots(
    1, 1, subplot_kw={'projection': map_proj})

da_EIGfilter.real.isel(time = int(5368/2)).plot.contourf(ax=axis, levels = 101,
                    transform=ccrs.PlateCarree())

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

In [ ]:
plt.rc("figure", figsize=(16,4))
plt.rc("font", size=14)
ccrs.PlateCarree()

map_proj = ccrs.Mercator(central_longitude=0.0, 
                         min_latitude=-30.0, 
                         max_latitude=30.0, 
                         globe=None)

fig, axis = plt.subplots(
    1, 1, subplot_kw={'projection': map_proj})

da_TDfilter.real.isel(time = int(5368/2)).plot.contourf(ax=axis, levels = 101,
                    transform=ccrs.PlateCarree())

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

In [ ]:
plt.rc("figure", figsize=(12,10))
plt.rc("font", size=14)

test = (_dahat_WIG.real*xu.conj(_dahat_WIG)).real.sum('lat')

In [ ]:
test.plot.contourf(x='freq_lon', y = 'freq_time', levels = 20,
                       cmap='Spectral_r', vmax = 10)